In [ ]:
pip install bottle

In [ ]:
from bottle import route, run

@route('/hola')
def hello():
    return "Hola Botella de coca de 1 litro"

run(host='0.0.0.0',port=5000,debug=True)

route() es un decorador que conecta una funcion a una URL, en este caso conectamos a la ruta /hola a la función hello(). Esto es llamado a una ruta, y es uno de los conceptos mas importantes de este framework. Se pueden definir tantas rutas como sean necesarias, y cuando desde un navegador solicite una URL, la función asociada a esa ruta será ejecutada y responderá al navegador.

La función run() en las ultimas lineas inicia el servidor web de desarrollo, este se ejecuta en local en el puerto 8080, y responde a las solicitudes hasta que presionamos ctrl-c. Podemos cambiar el servidor, esto en caso de llevar a "producción" nuestra aplicación, pero para desarrollo con las opciones default nos es suficiente, no necesita instalarse y es lo mas sencillo para probar nuestra aplicación.

## La aplicacion default

Podemos observar en el ejemplo que usamos el decorador route() para definir las rutas, esto añade las rutas a nuestra aplicación default, una instancia que se crea la primera vez que usamos el decorador route(), pero para seguir un enfoque mas orientado a objetos, crearemos primero la instancia de la aplicacion y añadiremos estas rutas a nuestra instancia explicitamente.

In [1]:
from bottle import Bottle, run

app=Bottle()

@app.route('/hola')
def hello():
    return "Hola Mundo"

run(app, host='0.0.0.0',port=5000, debug=True)

Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:5000/
Hit Ctrl-C to quit.

10.0.2.2 - - [15/Mar/2019 10:07:27] "GET /hola HTTP/1.1" 200 10
/home/vagrant/.local/share/virtualenvs/Jupyternotebooks-vMpxa5dg/lib/python3.6/site-packages/bottle.py:3131: ResourceWarning: unclosed <socket.socket fd=55, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 5000)>
  pass


## Añadiendo una ruta mas

Podemos añadir mas de una ruta a una función, además podemos añadir "comodines" (wildcars) a la ruta para hacerla más dinámica. Además veremos en acción el modulo template, donde nuestra función regresará un valor y será reemplazado en nuestro HTML.

In [2]:
from bottle import Bottle,run,template

app=Bottle()

@app.route('/')
@app.route('/hola/<name>')
def hello(name='Extraño'):
    return template('Hola {{label}}, cómo estas?',label=name)

run(app,host='0.0.0.0',port=5000,debug=True)

Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:5000/
Hit Ctrl-C to quit.

10.0.2.2 - - [15/Mar/2019 10:15:01] "GET / HTTP/1.1" 200 27
10.0.2.2 - - [15/Mar/2019 10:15:49] "GET /hello HTTP/1.1" 404 730
10.0.2.2 - - [15/Mar/2019 10:16:02] "GET /hola HTTP/1.1" 404 728
10.0.2.2 - - [15/Mar/2019 10:16:08] "GET /hola/Luis HTTP/1.1" 200 23
10.0.2.2 - - [15/Mar/2019 10:17:17] "GET /hola/Pinche%20putita HTTP/1.1" 200 32
10.0.2.2 - - [15/Mar/2019 10:17:25] "GET /hola/Luis HTTP/1.1" 200 23


## Un ejemplo de POST y GET

Podemos ver un ejemplo de manejar los diferentes verbos o métodos HTTP con el ejemplo de un login sencillo:

In [8]:
from bottle import Bottle,run,template,request,redirect

app=Bottle()

@app.route('/')
@app.route('/hola/<name>')
def hello(name='Extraño'):
    return template('Hola {{name}},como estas?',name=name)

@app.route('/loged/<name>')
def loged(name='Extraño'):
    return template('''You're in! {{name}} ''',name=name)

def checkLogin(username,password):
    isLogged=False
    if username=='Luis' and password=='123':
        isLogged=True
    return isLogged

@app.route('/login',method='GET')
def login():
    isFailed='0' if request.query.Failed== '' else request.query.Failed
    
    formString='''<form action="/login" method="post">
        Username: <input name="username" type="text" />
        Password: <input name="password" type="password" />
        <input value="Login" type="submit" />
      </form>  
    '''
    
    if int(isFailed)==1:
        return '<p><strong>Login failed!</p>'+formString
    else:
        return formString

@app.post('/login')# or @route('login',method='POST')
def do_login():
    username=request.forms.get('username')
    password=request.forms.get('password')
    if checkLogin(username,password):
        redirect(f"/loged/{username}")
    else:
        redirect("/login?Failed=1")

run(app,host='0.0.0.0',port=5000,debug=True)

Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:5000/
Hit Ctrl-C to quit.

10.0.2.2 - - [15/Mar/2019 10:54:02] "GET /login HTTP/1.1" 200 219
10.0.2.2 - - [15/Mar/2019 10:57:25] "POST /login HTTP/1.1" 303 0
10.0.2.2 - - [15/Mar/2019 10:57:25] "GET /loged/Luis HTTP/1.1" 200 16
10.0.2.2 - - [15/Mar/2019 10:57:58] "GET /login HTTP/1.1" 200 219
10.0.2.2 - - [15/Mar/2019 10:58:01] "POST /login HTTP/1.1" 303 0
10.0.2.2 - - [15/Mar/2019 10:58:01] "GET /login?Failed=1 HTTP/1.1" 200 247
10.0.2.2 - - [15/Mar/2019 10:58:17] "GET /login HTTP/1.1" 200 219
10.0.2.2 - - [15/Mar/2019 10:58:19] "POST /login HTTP/1.1" 303 0
10.0.2.2 - - [15/Mar/2019 10:58:19] "GET /loged/Luis HTTP/1.1" 200 16
10.0.2.2 - - [15/Mar/2019 10:59:39] "GET /loged/Fer HTTP/1.1" 200 15
